# Sparse NMF for Separation

One approach to using NMF for source separation is to learn sets of $k$ basis vectors $W$ and $H$ for each of the speakers in a dataset. To separate a mixture where the speakers are known, we concatenate the dictionaries $W$ associated with them, learn new loadings $H$, and take the product of the speaker-specific $W$ with the associated components of $H$ to yield the reconstruction.

Training, for speaker $i$:
$$X_i = W_i H_i$$
$$W_i, H_i = \text{NMF}(X_i)$$

Evaluation, on mixture $X_{ij}$ of speech from speaker $i$ and $j$. $\text{NMF}_W$ performs NMF updates without updating the values in $W$:
$$W_{ij} = [ W_i \, W_j ]$$
$$H_{ij}' = \text{NMF}_{W_{ij}}(X_{ij})$$
$$H_{ij}' = [ H_i' \, H_j' ]$$
$$\hat{X}_i = W_i H_i$$
$$\hat{X}_j = W_j H_j$$

In [ ]:
import sys 
from itertools import islice, permutations, product
from collections import namedtuple
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import IPython.display as display

from magnolia.features.hdf5_iterator import Hdf5Iterator
from magnolia.features.mixer import FeatureMixer
#from magnolia.features.wav_iterator import batcher
from magnolia.utils.tf_utils import scope_decorator as scope
from magnolia.utils.bss_eval import bss_eval_sources
from magnolia.factorization.nmf import snmf
from magnolia.features.spectral_features import reconstruct

num_srcs = 2
num_steps = 80
num_freq_bins = 257
num_components = 20
sparsity = 0.1

# librispeech_dev = "/local_data/teams/magnolia/librispeech/processed_dev-clean.h5"
# librispeech_train = "/local_data/teams/magnolia/librispeech/processed_train-clean-100.h5"
librispeech_dev = "/Users/patrickc/data/LibriSpeech/processed_dev-clean.h5"

def scale_spectrogram(spectrogram):
    mag_spec = np.abs(spectrogram)
    phases = np.unwrap(np.angle(spectrogram))
    
    mag_spec = np.sqrt(mag_spec)
    M = mag_spec.max()
    m = mag_spec.min()
    
    return (mag_spec - m)/(M - m), phases

def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

%matplotlib inline

## Data
### Get speaker-specific iterators

In [ ]:
with open("../../data/librispeech/authors/dev-clean-F.txt") as f:
    female_dev = f.read().strip().split('\n')

female_spkrs = [Hdf5Iterator(hdf5_path=librispeech_dev, speaker_keys=[dev], shape=(None,)) for dev in female_dev]
female_spkrs_slice = [Hdf5Iterator(hdf5_path=librispeech_dev, speaker_keys=[dev], shape=(num_steps,)) for dev in female_dev]

with open("../../data/librispeech/authors/dev-clean-M.txt") as f:
    male_dev = f.read().strip().split('\n')

male_spkrs = [Hdf5Iterator(hdf5_path=librispeech_dev, speaker_keys=[dev], shape=(None,)) for dev in male_dev]
male_spkrs_slice = [Hdf5Iterator(hdf5_path=librispeech_dev, speaker_keys=[dev], shape=(num_steps,)) for dev in male_dev]

## Training

In [ ]:
spkr_models = []
for i, spkr in enumerate(female_spkrs):
    print("Speaker", i)
    w = None
    for example in islice(spkr,45):
        mag, phases = scale_spectrogram(example)
        try:
            w, h = snmf(mag.T, num_components, sparsity=sparsity, num_iters=10, W_init=w)
        except ValueError:
            print("ValueError encountered", file=sys.stderr)
            continue
        
    plt.figure(figsize=(14,5))
    plt.subplot(1,3,1)
    plt.imshow((w @ h)[:,:100], cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,2)
    plt.imshow(mag.T[:,:100], cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,3)
    plt.imshow(w.T, cmap='bone', origin='lower', aspect=6)
    plt.show()
    spkr_models.append((w,h))

## Inference

In [ ]:
# Inference on women speakers
for i in range(num_components):
    spkr_i = np.random.randint(len(female_spkrs))
    spkr_j = np.random.randint(len(female_spkrs))
    if spkr_i == spkr_j:
        continue
    
    example_i = next(female_spkrs_slice[spkr_i])
    example_j = next(female_spkrs_slice[spkr_j])
    
    mix = example_i + example_j
    mix_scl_mag, mix_scl_phs = scale_spectrogram(mix)
    
    # combine dictionaries
    w_i, h_i = spkr_models[spkr_i]
    w_j, h_j = spkr_models[spkr_j]
    w_all = np.concatenate((w_i, w_j), axis=1)
    
    # train up
    _, h_new = snmf(mix_scl_mag.T, num_components*2, sparsity=sparsity, W=w_all)
    h_new_i = h_new[:num_components]
    h_new_j = h_new[num_components:]
    
    # separate
    reco_i = w_i @ h_new_i
    reco_j = w_j @ h_new_j
    
    plt.figure(figsize=(14,5))
    plt.subplot(1,3,1)
    plt.imshow(mix_scl_mag.T, cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,2)
    plt.imshow(reco_i, cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,3)
    plt.imshow(reco_j, cmap='bone', origin='lower', aspect=1/4)
    plt.show()
    
    display.display(display.Audio(reconstruct(reco_i.T**2, mix, 10000, None, 0.0256), rate=10000))
    display.display(display.Audio(reconstruct(reco_j.T**2, mix, 10000, None, 0.0256), rate=10000))

## Out-of-sample inference

Above technique only works when you know which set of basis vectors to select for each speaker.  For unseen speakers this is hard. One approach is just to pick the combination of dictionary entries that minimizes the reconstruction error. Unfortunately quadratic in the number of dictionary entries.

(Getting the cross-correlation of each basis with the mixture and picking the top two is another idea.)

In [ ]:
# Inference on men speakers
for i in range(5):
    print("Test {}".format(i))
    spkr_i = np.random.randint(len(male_spkrs))
    spkr_j = np.random.randint(len(male_spkrs))
    if spkr_i == spkr_j:
        continue
        
    example_i = next(male_spkrs_slice[spkr_i])
    example_j = next(male_spkrs_slice[spkr_j])

    mix = example_i + example_j
    mix_scl_mag, mix_scl_phs = scale_spectrogram(mix)

    # loop over choices of speaker model
    NmfSearchResult = namedtuple("NMFSearchResult", ['i', 'j', 'error', 'models', 'reconstructions'])
    optimal_pair = NmfSearchResult(0, 0, np.inf, [], [])  # i, j, error, reconstructions a and b
    for model_i in range(num_components):
        for model_j in range(model_i + 1, num_components):
            w_i, h_i = spkr_models[model_i]
            w_j, h_j = spkr_models[model_j]
            w_all = np.concatenate((w_i, w_j), axis=1)
            
            # train up
            _, h_new = snmf(mix_scl_mag.T, num_components*2, sparsity=sparsity, W=w_all)
            h_new_i = h_new[:num_components]
            h_new_j = h_new[num_components:]
            reco_i = w_i @ h_new_i
            reco_j = w_j @ h_new_j
            
            err = np.mean(np.abs(mix_scl_mag.T - reco_i + reco_j))
            
            if err < optimal_pair.error:
                optimal_pair = NmfSearchResult(model_i, model_j, err, [w_i, w_j], [reco_i, reco_j])
                
    # Display results
    print("Used weight matrices {} and {}".format(optimal_pair.i, optimal_pair.j))
    
    plt.figure(figsize=(14,5))
    plt.subplot(1,3,1)
    plt.imshow(mix_scl_mag.T, cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,2)
    plt.imshow(optimal_pair.reconstructions[0], cmap='bone', origin='lower', aspect=1/4)
    plt.subplot(1,3,3)
    plt.imshow(optimal_pair.reconstructions[1], cmap='bone', origin='lower', aspect=1/4)
    plt.show()
    
    # Evaluate
    ref_i = reconstruct(example_i, example_i, 10000, None, 0.0256)
    ref_j = reconstruct(example_j, example_j, 10000, None, 0.0256)
    opt_reco_i = reconstruct(optimal_pair.reconstructions[0].T, mix, 10000, None, 0.0256)
    opt_reco_j = reconstruct(optimal_pair.reconstructions[1].T, mix, 10000, None, 0.0256)

    print("SDR: {}, SIR: {}, SAR: {}, perm {}".format(*bss_eval_sources(np.stack([ref_i, ref_j]), 
                                                                       np.stack([opt_reco_i, opt_reco_j]))))
    
    display.display(display.Audio(reconstruct(mix, mix, 10000, None, 0.0256), rate=10000))
    display.display(display.Audio(opt_reco_i, rate=10000))
    display.display(display.Audio(opt_reco_j, rate=10000))  

In [ ]:
bss_eval_sources(np.stack([ref_i, ref_j]), 
                                                                       np.stack([opt_reco_i, opt_reco_j]))